In [1]:
%load_ext autotime
import geopandas as gpd # Geospatial data operations
import rasterio as rio # Geospatial imagery manipulation
import rasterio.plot
import pandas as pd # Tabular data
import os
import re
import rapidfuzz # Fuzzy string matching
from tqdm.auto import tqdm # Progress bars
from tqdm.contrib.concurrent import thread_map, process_map # Parallel operations
import matplotlib # Plots
import matplotlib.pyplot as plt
import shapely # Polygon operations
#import solaris.tile as tile # Tile splitting
#import solaris.data.coco as coco
import contextlib
import io
import rasterio # Raster imagery operations
from rasterio.vrt import WarpedVRT
from rasterio import transform
from rasterio.merge import merge # Merging tiles into mosaics
from glob import glob # Finding files
from shapely.geometry import box # Bounding box operations
matplotlib.rcParams['figure.figsize'] = (20, 10)
tqdm.pandas()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
import platform
if platform.system() == "Windows":
  prefix = "Z:/"
else:
  prefix = "ressci201900060-RNC2-Coastal/"

## Match shapefiles to images

In [2]:
# Set this to False to rebuild filelist.txt and meta.csv
use_cache = True

In [3]:
# This cell will read a list of files from Nick/filelist.txt if it exists, or build a list of files and save it to that location
# This is useful for quickly loading the list of files without having to search the entire directory structure
# You will need to delete the file and rerun this cell if files are created, renamed or moved

filename = prefix + "Nick/filelist.txt"
if use_cache and os.path.isfile(filename):
    filelist = pd.read_csv(filename, header=None).iloc[:,0]
else:
    def find_files(root):
        return pd.Series(glob(prefix + root + "**/**", recursive=True)).str.replace(prefix, "")
    filelist = pd.concat(thread_map(find_files, ["Gabrielle", "MaxarImagery", "Retrolens", "SpatialData/Mosaics"]))
    if platform.system() == "Windows":
        filelist = filelist.str.replace("\\", "/", regex=False)
    filelist.to_csv(filename, index=False, header=False)
filelist

0                                                                                       Gabrielle/
1                                                                                 Gabrielle/Orders
2                                                                            Gabrielle/Orders/AOIs
3                                          Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023WGS84.sbn
4                                               Gabrielle/Orders/AOIs/Pauanui_Tairua_07JAN2023.sbx
                                                    ...                                           
243531    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L02_R00000062_C00000065.tfw
243532    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L03_R00000020_C00000022.tfw
243533    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L01_R00000125_C00000131.tif
243534    SpatialData/Mosaics/Nelson.Overviews/NEL18R.Overviews/Ov_i03_L01_R00000127_C00000133.tif
243535    

In [4]:
def check_filename(filename):
    # This regex only matches shapefiles that contain something date-like in their names
    match = re.search(r'/Shorelines/.+\d{4}\w*.shp$', filename)
    return bool(match)

shapefiles = filelist[filelist.apply(check_filename)]
df = shapefiles.to_frame(name="filename")
df

,filename
29906,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_18FEB2023.shp
29907,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Matarangi_24DEC2022.shp
29910,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/NewChums_18FEB2023.shp
29911,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_24DEC2022.shp
29918,Gabrielle/Shorelines/Waikato/Matarangi and surrounds/Whangapoua_18FEB2023.shp
...,...
187338,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_02JAN1988.shp
187343,Retrolens/Wellington/KapitiSouth/Shorelines/KapitiSouth_06OCT1980.shp
188447,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_22AUG1961.shp
188458,Retrolens/Wellington/PukeruaBay/Shorelines/PukeruaBay_Wellington_13FEB2021.shp


In [5]:
images = filelist[filelist.str.contains("Stack", case=False) & filelist.str.endswith((".jpg", ".jp2", ".tif", ".TIF"))]
images

32627     MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_2.tif
32628     MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_08NOV2019_1.tif
32630       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_12MAR2018.tif
32633       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_31AUG2005.tif
32647       MaxarImagery/HighFreq/HawkesBay/Mahanga/Imagery/Stack/Mahanga_25DEC2015.tif
                                              ...                                      
188684            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_19NOV1972_mosaic.jp2
188690            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_04APR1986_mosaic.jp2
188693            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_10NOV1977_mosaic.tif
188694            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_22AUG1961_mosaic.tif
188697            Retrolens/Wellington/PukeruaBay/Stack/PukeruaBay_01AUG1942_mosaic.jp2
Name: 0, Length: 2619, dtype: ob

In [6]:
Gabrielle_images = filelist[filelist.str.startswith("Gabrielle") & filelist.str.endswith((".jpg", ".jp2", ".tif", ".TIF"))]

In [7]:
# When fuzzy matching, ignore these strings
# _0 will ignore leading zeros in dates
strings_to_delete = ["_mosaic", "_mosiac", "_mosaid", ".mosaic", "_cliff", "_beach", "_beachcliffsegment", "_MF.shp", "_MT.shp", "_0", "_1.tif", "_2.tif", "_3.tif", "_LDS", "_", " "]

def fuzz_preprocess(filename):
    for s in strings_to_delete:
        filename = filename.replace(s, "")
    # Case-insensitive
    filename = filename.lower()
    # Ignore extension
    filename = os.path.splitext(filename)[0]
    # Basename only
    filename = os.path.basename(filename)
    return filename

def get_matching_image(filename):
    if filename.startswith("Gabrielle"):
        match, score, index = rapidfuzz.process.extractOne(query=filename, choices=Gabrielle_images, processor=fuzz_preprocess)
        return match, score
    else:
        match, score, index = rapidfuzz.process.extractOne(query=filename, choices=images, processor=fuzz_preprocess)
        return match, score

df["matched_image"], df["match_score"] = zip(*process_map(get_matching_image, df.filename))
print("Perfect matches:", sum(df.match_score == 100))
print("Imperfect matches:", sum(df.match_score < 100))

/tmp/ipykernel_2844253/4132008065.py:24: TqdmWarning: Iterable length 2444 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  df["matched_image"], df["match_score"] = zip(*process_map(get_matching_image, df.filename))


  0%|          | 0/2444 [00:00<?, ?it/s]

Perfect matches: 2012
Imperfect matches: 432


In [8]:
df = df.sort_values("match_score")
df

,filename,matched_image,match_score
30274,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Opotiki/Opotiki_28FEB2023.tif,57.894737
31160,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Onetangi_21DEC2022.tif,58.823529
30441,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_20DEC2021.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Matarangi_24DEC2022.tif,59.459459
30213,Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Omaha/PNEO/OmahaPakiri_04APR2023.tif,61.538462
122120,Retrolens/Southland/Riverton/Shorelines/Riverton_Mosaic_STH13R_05FEB2014.shp,MaxarImagery/HighFreq/Southland/Riverton/Stack/Riverton_13MAR2021.tif,62.307692
...,...,...,...
81557,Retrolens/HawkesBay/Porangahau/Shorelines/Porangahau_07SEP1944.shp,Retrolens/HawkesBay/Porangahau/Stack/Porangahau_07SEP1944_mosaic.jp2,100.000000
81563,Retrolens/HawkesBay/Porangahau/Shorelines/Porangahau_23NOV1976.shp,Retrolens/HawkesBay/Porangahau/Stack/Porangahau_23NOV1976_mosaic.jp2,100.000000
81779,Retrolens/HawkesBay/Wairoa/Shorelines/Wairoa_22JAN1979.shp,Retrolens/HawkesBay/Wairoa/Stack/Wairoa_22JAN1979_mosaic.jp2,100.000000
81366,Retrolens/HawkesBay/WhakakiLagoon/Shorelines/WhakakiLagoon_26SEP1970.shp,Retrolens/HawkesBay/WhakakiLagoon/Stack/WhakakiLagoon_26SEP1970_mosaic.jp2,100.000000


In [9]:
def get_shapefile_meta(tup):
    i, row = tup
    filename = row.filename
    source = "Unknown"
    try:
        shapefile = gpd.read_file(prefix+filename)
    except:
        print(f"Can't read {filename}")
        return {"Source": source, "n_lines": 0}
    if "Source" not in shapefile.columns:
        if filename.startswith("Retrolens"):
            source = "RL"
        elif filename.startswith("MaxarImagery/HighFreq"):
            source = "MAX"
    else:
        sources = shapefile.Source.unique()
        if len(sources) == 0 or not sources[0]:
            #print(f"{filename} has no sources")
            if filename.startswith("MaxarImagery/HighFreq"):
                source = "MAX"
        elif len(sources) == 1:
            source = sources[0]
        elif len(sources) > 1:
            print(f"{filename} has ambiguous sources: {sources}")
            source = sources[0]
    n_lines = len(shapefile.dropna(subset="geometry").explode(index_parts=False))
    return {
        "Source": source,
        "n_lines": n_lines,
        "CPS": "CPS" in shapefile.columns,
        "Photoscale": "Photoscale" in shapefile.columns,
        "Pixel_Er": "Pixel_Er" in shapefile.columns
    }

shp_meta = pd.DataFrame(process_map(get_shapefile_meta, df.iterrows(), total=len(df)), index=df.index)
df = pd.concat([df, shp_meta], axis=1)
df

  0%|          | 0/2444 [00:00<?, ?it/s]

Retrolens/HawkesBay/Awatoto/Shorelines/Awatoto_06MAR2019.shp has ambiguous sources: ['LDS' None]
Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp has ambiguous sources: ['LDS' None]
Retrolens/Taranaki/OpunakeBeach/Shorelines/OpunakeBeach_11FEB2017.shp has ambiguous sources: ['LDS' None]
Retrolens/Otago/Warrington/Shorelines/Warrington_17MAR2000.shp has ambiguous sources: ['RL' None]
Retrolens/Tasman/MoutereRiver/Shorelines/MoutereRiver_13SEP1985.shp has ambiguous sources: ['RL' None]
Can't read Retrolens/Canterbury/Washdyke/Shorelines/Washdyke_19JAN1999.shp
Retrolens/WestCoast/Westport/Shorelines/Westport_15JAN1988.shp has ambiguous sources: ['Rl' 'RL']
Retrolens/Southland/HaldaneBay/Shorelines/HaldaneBay_07OCT1985.shp has ambiguous sources: ['RL' None]
Retrolens/Auckland/PakiriBeach/Shorelines/PakiriBeach_24OCT1953.shp has ambiguous sources: ['RL' None]
MaxarImagery/HighFreq/Wellington/KapitiMid/Shorelines/KapitiMid_31OCT2019.shp has ambiguous sources: ['MAX' None]
MaxarI

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30274,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Opotiki/Opotiki_28FEB2023.tif,57.894737,LDS,2,True,True,True
31160,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Onetangi_21DEC2022.tif,58.823529,LDS,48,True,True,True
30441,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_20DEC2021.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Matarangi_24DEC2022.tif,59.459459,LDS,34,True,True,True
30213,Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Omaha/PNEO/OmahaPakiri_04APR2023.tif,61.538462,LDS,6,True,True,True
122120,Retrolens/Southland/Riverton/Shorelines/Riverton_Mosaic_STH13R_05FEB2014.shp,MaxarImagery/HighFreq/Southland/Riverton/Stack/Riverton_13MAR2021.tif,62.307692,LDS,8,True,False,False
...,...,...,...,...,...,...,...,...
81557,Retrolens/HawkesBay/Porangahau/Shorelines/Porangahau_07SEP1944.shp,Retrolens/HawkesBay/Porangahau/Stack/Porangahau_07SEP1944_mosaic.jp2,100.000000,RL,4,True,True,True
81563,Retrolens/HawkesBay/Porangahau/Shorelines/Porangahau_23NOV1976.shp,Retrolens/HawkesBay/Porangahau/Stack/Porangahau_23NOV1976_mosaic.jp2,100.000000,RL,2,True,True,True
81779,Retrolens/HawkesBay/Wairoa/Shorelines/Wairoa_22JAN1979.shp,Retrolens/HawkesBay/Wairoa/Stack/Wairoa_22JAN1979_mosaic.jp2,100.000000,RL,8,True,True,True
81366,Retrolens/HawkesBay/WhakakiLagoon/Shorelines/WhakakiLagoon_26SEP1970.shp,Retrolens/HawkesBay/WhakakiLagoon/Stack/WhakakiLagoon_26SEP1970_mosaic.jp2,100.000000,RL,6,True,True,True


In [10]:
df.sort_values(by="match_score")

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30274,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Opotiki/Opotiki_28FEB2023.tif,57.894737,LDS,2,True,True,True
31160,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Onetangi_21DEC2022.tif,58.823529,LDS,48,True,True,True
30441,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_20DEC2021.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Matarangi_24DEC2022.tif,59.459459,LDS,34,True,True,True
30213,Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Omaha/PNEO/OmahaPakiri_04APR2023.tif,61.538462,LDS,6,True,True,True
122120,Retrolens/Southland/Riverton/Shorelines/Riverton_Mosaic_STH13R_05FEB2014.shp,MaxarImagery/HighFreq/Southland/Riverton/Stack/Riverton_13MAR2021.tif,62.307692,LDS,8,True,False,False
...,...,...,...,...,...,...,...,...
165080,Retrolens/Northland/BayleysBeach/Shorelines/BayleysBeach_05FEB1996.shp,Retrolens/Northland/BayleysBeach/Stack/BayleysBeach_05FEB1996.tif,100.000000,RL,6,True,True,True
164990,Retrolens/Northland/Whananaki/Shorelines/Whananaki_29MAR1951.shp,Retrolens/Northland/Whananaki/Stack/Whananaki_29MAR1951.tif,100.000000,RL,25,True,True,True
164988,Retrolens/Northland/Whananaki/Shorelines/Whananaki_17MAR1961.shp,Retrolens/Northland/Whananaki/Stack/Whananaki_17MAR1961.tif,100.000000,RL,8,True,True,True
164954,Retrolens/Northland/Whananaki/Shorelines/Whananaki_22DEC1968.shp,Retrolens/Northland/Whananaki/Stack/Whananaki_22DEC1968.tif,100.000000,RL,21,True,True,False


In [11]:
df.to_csv(prefix+"Nick/shoreline_image_matching.csv", index=False)

### For those perfect matches, get the metadata from the corresponding matched image

In [12]:
# Investigate metadata about the matched images

def get_meta(tup):
    i, row = tup
    try:
        image = rio.open(prefix + row.matched_image)
    except:
        print(f"Can't read {row.matched_image}")
        return {}
    try:
        gdf = gpd.read_file(prefix + row.filename)
        row = row.to_dict()
        row["n_lines"] = len(gdf.dropna(subset="geometry").explode(index_parts=False))
    except: 
        print(f"Can't read {row['filename']}")
        return {}
    
    row.update(image.profile)
    row["GCPs"] = len(image.gcps[0])
    row["res"] = image.res
    row["CPS"] = "CPS" in gdf.columns
    return row

metafile = "meta.csv"
if use_cache and os.path.isfile(metafile):
    meta = pd.read_csv(metafile)
else:
    meta = pd.DataFrame(process_map(get_meta, df[df.match_score == 100].iterrows(), total=sum(df.match_score == 100)))
    meta.to_csv(metafile, index=False)
meta

  0%|          | 0/2012 [00:00<?, ?it/s]

Can't read Retrolens/Canterbury/Washdyke/Shorelines/Washdyke_19JAN1999.shp


,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er,driver,dtype,nodata,width,height,count,crs,transform,blockxsize,blockysize,tiled,interleave,GCPs,res,compress,photometric
0,Retrolens/Otago/NorthOamaru/Shorelines/NorthOamaru_26OCT1966.shp,Retrolens/Otago/NorthOamaru/Stack/NorthOamaru_26OCT1966_mosaic.jp2,100.0,RL,5.0,True,True,True,JP2OpenJPEG,uint16,256.0,8555.0,12445.0,3.0,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.4924023160382437, 0.0, 1450154.0302979634, 0.0, -0.4924023160382405, 5022485.608172786, 0.0, 0.0, 1.0)",1024.0,1024.0,True,pixel,0.0,"(0.4924023160382437, 0.4924023160382405)",NaN,NaN
1,Retrolens/Manawatu/Herbertville/Shorelines/Herbertville_17FEB1995.shp,Retrolens/Manawatu/Herbertville/Stack/Herbertville_17FEB1995_mosaic.jp2,100.0,RL,1.0,True,False,False,JP2OpenJPEG,uint16,256.0,5980.0,3180.0,3.0,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(1.5, 0.0, 1895301.9698752838, 0.0, -1.5, 5511406.97438402, 0.0, 0.0, 1.0)",1024.0,1024.0,True,pixel,0.0,"(1.5, 1.5)",NaN,NaN
2,Retrolens/Otago/Taieri_Brighton/Shorelines/Taieri_Brighton_03NOV1995.shp,Retrolens/Otago/Taieri_Brighton/Stack/Taieri_Brighton_03NOV1995_mosaic.jp2,100.0,RL,8.0,True,True,True,JP2OpenJPEG,uint16,256.0,4244.0,4697.0,3.0,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(1.9892396837772786, 0.0, 1385523.0122911718, 0.0, -1.9892396837772715, 4908822.504666761, 0.0, 0.0, 1.0)",1024.0,1024.0,True,pixel,0.0,"(1.9892396837772786, 1.9892396837772715)",NaN,NaN
3,Retrolens/Otago/NorthOamaru/Shorelines/NorthOamaru_09MAY1955.shp,Retrolens/Otago/NorthOamaru/Stack/NorthOamaru_09MAY1955_mosaic.jp2,100.0,RL,3.0,True,True,True,JP2OpenJPEG,uint16,256.0,17865.0,26846.0,3.0,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.49811576733726926, 0.0, 1445198.3788442442, 0.0, -0.49811576733727553, 5022177.283355617, 0.0, 0.0, 1.0)",1024.0,1024.0,True,pixel,0.0,"(0.49811576733726926, 0.49811576733727553)",NaN,NaN
4,Retrolens/Manawatu/SantoftForestMid/Shorelines/SantoftForestMid_23MAY1949.shp,Retrolens/Manawatu/SantoftForestMid/Stack/SantoftForestMid_23MAY1949_mosaic.jp2,100.0,RLN,4.0,True,True,True,JP2OpenJPEG,uint16,256.0,4182.0,13661.0,3.0,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.5999999999999889, 0.0, 1785808.3518291672, 0.0, -0.5999999999999728, 5553838.849840027, 0.0, 0.0, 1.0)",1024.0,1024.0,True,pixel,0.0,"(0.5999999999999889, 0.5999999999999728)",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007,Retrolens/HawkesBay/Porangahau/Shorelines/Porangahau_07SEP1944.shp,Retrolens/HawkesBay/Porangahau/Stack/Porangahau_07SEP1944_mosaic.jp2,100.0,RL,4.0,True,True,True,JP2OpenJPEG,uint16,NaN,4740.0,10760.0,3.0,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(0.4940602487347576, 0.0, 1910913.6982235978, 0.0, -0.4940602487347826, 5536328.318330653, 0.0, 0.0, 1.0)",1024.0,1024.0,True,pixel,0.0,"(0.4940602487347576, 0.4940602487347826)",NaN,NaN
2008,Retrolens/HawkesBay/Porangahau/Shorelines/Porangahau_23NOV1976.shp,Retrolens/HawkesBay/Porangahau/Stack/Porangahau_23NOV1976_mosaic.jp2,100.0,RL,2.0,True,True,True,JP2OpenJPEG,uint16,256.0,7682.0,6340.0,3.0,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(1.832847437903283, 0.0, 1908954.4789106173, 0.0, -1.8328474379032968, 5542343.150576575, 0.0, 0.0, 1.0)",1024.0,1024.0,True,pixel,0.0,"(1.832847437903283, 1.8328474379032968)",NaN,NaN
2009,Retrolens/HawkesBay/Wairoa/Shorelines/Wairoa_22JAN1979.shp,Retrolens/HawkesBay/Wairoa/Stack/Wairoa_22JAN1979_mosaic.jp2,100.0,RL,8.0,True,True,True,JP2OpenJPEG,uint16,256.0,4931.0,2249.0,3.0,"(proj, lat_0, lon_0, k, x_0, y_0, ellps, towgs84, units, no_defs)","(1.7068665583935856, 0.0, 1978283.8330965773, 0.0, -1.7068665583935605, 5668895.284172818, 0.0, 0.0, 1.0)",1024.0,1024.0,True,pixel,0.0,"(1.7068665583935856, 1.7068665583935605)",NaN,NaN
2010,Retrol

## Read LDS index tiles

In [13]:
index_tiles = filelist[filelist.str.contains(".+index-tiles.+.shp$")]
index_tiles

4515      Gabrielle/Imagery/post_storm/LINZ/HawkesBay/hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index-tiles-Copy.shp
10077             Gabrielle/Imagery/post_storm/LINZ/BayofPlenty/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2023.shp
11954          Gabrielle/Imagery/post_storm/LINZ/Gisborne/gisborne-02m-cyclone-gabrielle-aerial-photos-index-tiles-202.shp
13934          Gabrielle/Imagery/pre-storm/Waikato/TairuaPauanui/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
14053               Gabrielle/Imagery/pre-storm/Waikato/LINZtemp/waikato-03m-rural-aerial-photos-index-tiles-2021-2023.shp
                                                                ...                                                       
224423             SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-03m-rural-aerial-photos-index-tiles-2019.shp
224428        SpatialData/Mosaics/Bay of Plenty/Footprints/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2018-2019.shp
232033          

In [14]:
def read_index_tile(f):
    gdf = gpd.read_file(prefix+f).to_crs(2193)
    gdf["filename"] = f
    return gdf

index_tiles = pd.concat(process_map(read_index_tile, index_tiles))
len(index_tiles)

  0%|          | 0/79 [00:00<?, ?it/s]

147407

In [15]:
index_tiles.Captured = index_tiles.Captured.replace({"05, 06, 09, 10 Feb. 2014": "2014-02-05", "11 & 26 March 2014": "2014-03-11"})
index_tiles["parsed_date"] = pd.NA
for col in ['Date_Flown', 'Date_Suppl', 'DATE', 'DATE_DMY', 'FLOWN_DATE', 'FLY_DATE', 'ACQ_DATE', "FLYING_DAT", "FLOWN", "Captured"]:
    index_tiles["parsed_date"].update(pd.to_datetime(index_tiles[col].str.split(",|-|to").str[0], dayfirst=True, format="mixed", errors="ignore"))
index_tiles.parsed_date = index_tiles.parsed_date.astype("string")
index_tiles.parsed_date.value_counts(dropna=False)

/home/nyou045/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:1538: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)


parsed_date
2017                   23649
<NA>                   22679
2022                   11348
2023                    3607
24/02/2017              2685
                       ...  
9/11/2016                  1
2029-03-12 13:01:12        1
2005-05-12 12:05:22        1
2016-11-15 15:12:28        1
2028-11-15 16:03:27        1
Name: count, Length: 384, dtype: Int64

## LDS index tile matching

In [16]:
LDS = df[(df.Source == "LDS") & (df.match_score < 100)].sort_values("match_score")
LDS

,filename,matched_image,match_score,Source,n_lines,CPS,Photoscale,Pixel_Er
30274,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/BayofPlenty/Opotiki/Opotiki_28FEB2023.tif,57.894737,LDS,2,True,True,True
31160,Gabrielle/Shorelines/Gisborne/TeAraroa/EastCape_18DEC2021.shp,Gabrielle/Imagery/pre-storm/Auckland/Waiheke/Onetangi_21DEC2022.tif,58.823529,LDS,48,True,True,True
30441,Gabrielle/Shorelines/BayofPlenty/EasternBoP/EasternBoP_20DEC2021.shp,Gabrielle/Imagery/pre-storm/Waikato/Matarangi/Matarangi_24DEC2022.tif,59.459459,LDS,34,True,True,True
30213,Gabrielle/Shorelines/BayofPlenty/Waihi/BOPLINZ_Waihi_05APR2023.shp,Gabrielle/Imagery/post_storm/Region/Auckland/Omaha/PNEO/OmahaPakiri_04APR2023.tif,61.538462,LDS,6,True,True,True
122120,Retrolens/Southland/Riverton/Shorelines/Riverton_Mosaic_STH13R_05FEB2014.shp,MaxarImagery/HighFreq/Southland/Riverton/Stack/Riverton_13MAR2021.tif,62.307692,LDS,8,True,False,False
...,...,...,...,...,...,...,...,...
138370,Retrolens/Manawatu/CastleCliff/Shorelines/CastleCliff_10JUN1942_clifftop.shp,Retrolens/Manawatu/CastleCliff/Stack/CastleCliff_10JUN1942_mosaic.jp2,93.023256,LDS,1,True,False,False
155556,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Shorelines/Ryans_Pipikaretu_Penguin_TeRauoneBeach_01APR2006.shp,Retrolens/Otago/Ryans_Pipikaretu_Penguin_TeRauoneBeach/Stack/Ryans_Pipikaretu_Penguin_TeRauoneBeach_17MAR2000_mosaic.jp2,94.252874,LDS,8,True,False,False
117350,Retrolens/Southland/Tiwai_Point/Shorelines/TiwaiPoint_LDS_07FEB2016.shp,MaxarImagery/HighFreq/Southland/TiwaiPoint/Stack/TiwaiPoint_02FEB2016.tif,94.444444,LDS,1,True,False,False
125984,Retrolens/Taranaki/Oakura/Shorelines/Oakura_31OCT2016_cliff.shp,MaxarImagery/HighFreq/Taranaki/Oakura/Stack/Oakura_03OCT2016.tif,96.551724,LDS,5,True,False,False


In [17]:
def get_resolution(filename):
  gdf = gpd.read_file(prefix+filename)
  if "LDS" not in gdf.Source.unique():
    return {"filename": filename}
  bounds = gdf.total_bounds
  intersecting_tiles = index_tiles[index_tiles.intersects(box(*bounds))]
  if filename.startswith("Gabrielle"):
    intersecting_tiles = intersecting_tiles[intersecting_tiles.filename.str.startswith("Gabrielle")]
  if len(intersecting_tiles) == 0:
    print(f"{filename} doesn't intersect any index tiles")
    return {"filename": filename}
  if "Date" not in gdf.columns:
    #print(f"{filename} missing Date column")
    date = gdf.DSASdate.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  elif "DSASDate" not in gdf.columns:
    #print(f"{filename} missing DSASDate column")
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASdate.unique()[0]
  else:
    date = gdf.Date.unique()[0]
    DSASdate = gdf.DSASDate.unique()[0]
  if not DSASdate:
    DSASdate = date

  date_options = []
  date_to_col = {}
  for col in ['Date_Flown', 'Date_Suppl', 'DATE', 'DATE_DMY', 'FLOWN_DATE', 'FLY_DATE', 'ACQ_DATE', "FLYING_DAT", "FLOWN", "Captured"]:#, "parsed_date"]:
    options_for_col = intersecting_tiles[col].dropna().unique().tolist()
    date_options.extend(options_for_col)
    for date_option in options_for_col:
      date_to_col[date_option] = col
  display(date, DSASdate, gdf, intersecting_tiles.dropna(axis=1, how="all"), date_options, date_to_col)
  if not date_options:
    if "hawkes-bay-010m-cyclone-gabrielle-aerial-photos-index" in intersecting_tiles.filename.unique()[0]:
      return {
        "filename": filename,
        "matched_index_tiles": intersecting_tiles.filename.unique(),
        "date": date,
        "matched_date": "SPECIAL_OVERRIDE",
        "match_score": 100,
        "Pixel_ER": .1
      }
    else:
      print(f"No date options in {intersecting_tiles.filename.unique()}")
      return {"filename": filename}
  else:
    match = False
    if DSASdate in date_options:
      match = DSASdate
      score = 100
    for option in date_options:
      if DSASdate in option or date in option:
        match = option
        score = 100
    if not match:
      match, score, index = rapidfuzz.process.extractOne(query=date, choices=date_options, processor=lambda s: s.replace("-", ""))
    col_for_match = date_to_col[match]
    tiles_from_this_date = intersecting_tiles[intersecting_tiles[col_for_match] == match]
    GSDM = []
    for col in ['GSDM', 'ORTHO_GSD', 'Ortho_GSD', 'GSDm', 'GSD', 'GSD_M', 'GSD_CM', 'gsdM']:
      GSDM.extend(tiles_from_this_date[col].dropna().astype(str).str.strip("m").unique())
    if len(GSDM) == 0:
      tilefile = tiles_from_this_date.filename.unique()[0]
      if "-04m" in tilefile:
        GSDM = .4
      elif "-0075m" in tilefile:
        GSDM = .075
    elif len(GSDM) == 1:
      GSDM = GSDM[0]
    elif len(GSDM) > 1:
      #print(f"Ambiguous GSDM: {GSDM}")
      GSDM = GSDM[0]
    return {
      "filename": filename,
      "matched_index_tiles": tiles_from_this_date.filename.unique().tolist(),
      "date": date,
      "DSASDate": DSASdate,
      "matched_date": match,
      "matched_date_col": col_for_match,
      "match_score": score,
      "Pixel_ER": GSDM
    }

#get_resolution("Retrolens/Taranaki/PariokariwaPoint/Shorelines/PariokariwaPoint_19MAR2017_Taranaki399_cliff.shp")
LDS = pd.DataFrame(process_map(get_resolution, LDS.filename)).sort_values("match_score")
print("Perfect matches:", sum(LDS.match_score == 100))
print("Imperfect matches:", sum(LDS.match_score < 100))
LDS

  0%|          | 0/370 [00:00<?, ?it/s]

Perfect matches: 266
Imperfect matches: 104


,filename,matched_index_tiles,date,DSASDate,matched_date,matched_date_col,match_score,Pixel_ER
43,Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2015-01-09,09/01/2015,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",FLOWN_DATE,30.000000,0.3
171,Retrolens/Canterbury/KaitoreteSpitWest/Shorelines/KaitoreteSpitWest_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2003-04-22,22/04/2003,151125 151228,DATE,33.750000,0.3
129,Retrolens/Canterbury/KaitoreteSpitEast/Shorelines/KaitoreteSpitEast_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2023-04-22,22/04/2023,151228 151229,DATE,38.095238,0.3
334,Retrolens/Canterbury/Washdyke/Shorelines/Washdyke_14FEB2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-02-28,28/02/2017,14/02/17 to 09/03/17,FLOWN,42.750000,0.3
313,Retrolens/Canterbury/WainononLagoon_PareoaRiver/Shorelines/WainonoLagoon_PareoraRiver_02DEC2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-12-02,02/12/2017,02/12/17 to 08/02/18,FLOWN,42.750000,0.3
...,...,...,...,...,...,...,...,...
157,Retrolens/Taranaki/Kakaramea/Shorelines/Kakaramea_21NOV2017.shp,[SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-03m-rural-aerial-photos-index-tiles-2016-2018.shp],2017-11-21,21/11/2017,21/11/2017,FLOWN,100.000000,0.3
155,Retrolens/Gisborne/Whangara/Shorelines/Whangara_15FEB2018.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-gisborne-03m-rural-aerial-photos-index-tiles-2017-2019-SHP (1)/gisborne-03m-rural-aerial-photos-index-tiles-2017-2019.shp],2018-02-15,15/02/2018,15/02/2018,FLOWN,100.000000,0.3
154,Retrolens/Southland/Riverton/Shorelines/Riverton_19SEP2007.shp,"[Retrolens/Southland/Riverton/LDS/southland-075m-rural-aerial-photos-index-tiles-2005-2011.shp, SpatialData/Mosaics/Southland 2005-2011/Imagery/southland-075m-rural-aerial-photos-index-tiles-2005-...",2007-09-19,19/09/2007,19/09/2007,Date_Flown,100.000000,0.75
174,Retrolens/Marlborough/WairauRiver/Shorelines/WairauRiver_10NOV2021.shp,[SpatialData/Mosaics/Marlborough2021/marlborough-025m-rural-aerial-photos-index-tiles-2021-2022.shp],2021-11-10,10/11/2021,"10/11/2021, 08/11/2021, 10/11/2021",FLOWN,100.000000,0.25


In [18]:
with pd.option_context("display.max_rows", None):
  display(LDS[LDS.match_score < 100])

,filename,matched_index_tiles,date,DSASDate,matched_date,matched_date_col,match_score,Pixel_ER
43,Retrolens/Canterbury/Motunau/Shorelines/Motunau_09JAN2015.shp,[SpatialData/Mosaics/Footprint shapefiles/Kaikoura/kaikoura-030m-rural-aerial-photos-index-tiles-2016-2017.shp],2015-01-09,09/01/2015,"20/12/2016,21/12/2016,15/02/2017,16/02/2017,21/02/2017,24/02/2017",FLOWN_DATE,30.000000,0.3
171,Retrolens/Canterbury/KaitoreteSpitWest/Shorelines/KaitoreteSpitWest_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2003-04-22,22/04/2003,151125 151228,DATE,33.750000,0.3
129,Retrolens/Canterbury/KaitoreteSpitEast/Shorelines/KaitoreteSpitEast_22APR2023.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2015-2016-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2015-2016.shp],2023-04-22,22/04/2023,151228 151229,DATE,38.095238,0.3
334,Retrolens/Canterbury/Washdyke/Shorelines/Washdyke_14FEB2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-02-28,28/02/2017,14/02/17 to 09/03/17,FLOWN,42.750000,0.3
313,Retrolens/Canterbury/WainononLagoon_PareoaRiver/Shorelines/WainonoLagoon_PareoraRiver_02DEC2017.shp,[SpatialData/Mosaics/Footprint shapefiles/lds-canterbury-03m-rural-aerial-photos-index-tiles-2017-2018-SHP/canterbury-03m-rural-aerial-photos-index-tiles-2017-2018.shp],2017-12-02,02/12/2017,02/12/17 to 08/02/18,FLOWN,42.750000,0.3
0,Gabrielle/Shorelines/BayofPlenty/Opotiki/BOPLINZ_Opotiki_05APR2023.shp,[Gabrielle/Imagery/post_storm/LINZ/BayofPlenty/bay-of-plenty-01m-urban-aerial-photos-index-tiles-2023.shp],05/04/2023,05/04/2023,2023-04-05,FLOWN,44.444444,0.10
248,Retrolens/HawkesBay/OceanBeach/Shorelines/OceanBeach_06MAR2019.shp,[SpatialData/Mosaics/Footprint shapefiles/Hawkes Bay Footprints/hawkes-bay-03m-rural-aerial-photos-index-tiles-2019-2020.shp],2019-03-06,6/03/2019,05/03/2019,FLOWN,44.444444,0.3
365,Retrolens/Manawatu/CastleCliff/Shorelines/CastleCliff_10JUN1942_clifftop.shp,"[SpatialData/Mosaics/Footprint shapefiles/Manawatu/manawatu-whanganui-03m-rural-aerial-photos-index-tiles-2021-.shp, SpatialData/Mosaics/Manawatu 2021/manawatu-whanganui-03m-rural-aerial-photos-in...",1942-06-10,10/06/1942,"17/03/2021, 21/02/2021",FLOWN,45.000000,0.3
287,Retrolens/Otago/Warrington/Shorelines/Warrington_30MAR2006.shp,[SpatialData/Mosaics/Footprint shapefiles/Otago footprints/lds-dunedin-01m-urban-aerial-photos-index-tiles-2018-2019-SHP/dunedin-01m-urban-aerial-photos-index-tiles-2018-2019.shp],1899-12-30,1899-12-30,"07/04/2019,08/02/2019,06/11/2018,14/01/18",FLOWN,45.000000,0.1
178,Retrolens/Taranaki/UrenuiRiver_North_AOI/Shorelines/UrenuiRiver_North_19MARC2017_Taranaki399.shp,[SpatialData/Mosaics/Footprint shapefiles/Taranaki Footprints/taranaki-03m-rural-aerial-photos-index-tiles-2016-2018.shp],2017-03-19,19/03/2017,19/3/2017,FLOWN,47.058824,0.3


In [19]:
# Stop Excel interpreting dates
for col in ["date", "DSASDate", "matched_date"]:
    LDS[col] = "'" + LDS[col]
LDS.to_csv(prefix + "Nick/LDS_matches.csv", index=False)